In [ ]:
import os 
import importlib

from waffles.input_output.hdf5_structured import load_structured_waveformset
from waffles.data_classes.WaveformSet import WaveformSet
from waffles.data_classes.Waveform import Waveform
from waffles.np04_data.ProtoDUNE_HD_APA_maps import APA_map # apa map
from waffles.data_classes.ChannelWsGrid import ChannelWsGrid # channel grid

import integral_computation_function as integral_module
from integral_computation_function import channel_integral_computation

import waffles.np04_analysis.lightyield_vs_energy.scripts.utils_plots as utils_plots_module
from waffles.np04_analysis.lightyield_vs_energy.scripts.utils_plots import *


from waffles.np04_analysis.lightyield_vs_energy.scripts.MyAnaPeak import MyAnaPeak

In [ ]:
run_folder = "/afs/cern.ch/work/a/anbalbon/public/reading_rucio/run027361/full_streaming/no_selection"
fname = "processed_np04hd_raw_run027361_0001_dataflow0_datawriter_0_20240622T004415.hdf5_structured.hdf5"
filepath = os.path.join(run_folder, fname)
FS_wfset = load_structured_waveformset(filepath)  # load HDF5 structured waveform set
            

In [ ]:
importlib.reload(utils_plots_module)
from waffles.np04_analysis.lightyield_vs_energy.scripts.utils_plots import *
plotting_overlap_wf_PEAK_NEW(FS_wfset, n_wf= 20, show  = True, peak_bool = False, peak_beam_bool = False, inversion = False, input_signal_sign = "negative")

In [ ]:
apa = 1
endpoint = 104
channel = 7
grid_apa_FS = ChannelWsGrid(APA_map[apa], FS_wfset, compute_calib_histo=False)
mean_wf = grid_apa_FS.ch_wf_sets[endpoint][channel].compute_mean_waveform()
fig = go.Figure()
y = mean_wf.adcs
x = list(range(len(mean_wf.adcs)))
fig.add_trace(go.Scatter(x=x, y=y, mode='lines', name='mean_wf'))

fig.update_layout(
    title='Mean Waveform',
    xaxis_title='Samples',
    yaxis_title='Amplitude',
    template='plotly_white'
)

fig.show()

In [ ]:
from waffles.data_classes.IPDict import IPDict
from waffles.utils.baseline.WindowBaseliner import WindowBaseliner
baseliner_input_parameters = IPDict({'baseline_limits': [0,50], 'std_cut': 3., 'type': 'mean'})    
checks_kwargs = IPDict({'points_no': FS_wfset.points_per_wf})
_ = FS_wfset.analyse('baseliner', WindowBaseliner, baseliner_input_parameters, checks_kwargs=checks_kwargs, overwrite=True)


In [ ]:
from waffles.utils.integral.integral_utils import get_pulse_window_limits
limits_meanwf = get_pulse_window_limits(y, 690, 0.3, 0, 0)

In [ ]:
fig = go.Figure()
y = mean_wf.adcs
x = list(range(len(mean_wf.adcs)))
fig.add_trace(go.Scatter(x=x, y=y, mode='lines', name='mean_wf'))

fig.update_layout(
    title='Mean Waveform',
    xaxis_title='Samples',
    yaxis_title='Amplitude',
    template='plotly_white'
)

fig.add_shape(
    type="line",
    x0=limits_meanwf[0], x1=limits_meanwf[0],
    y0=min(y), y1=max(y),
    line=dict(width=2, dash="dash", color="red")
)

fig.add_shape(
    type="line",
    x0=limits_meanwf[1], x1=limits_meanwf[1],
    y0=min(y), y1=max(y),
    line=dict(width=2, dash="dash", color="red")
)

fig.show()

In [ ]:
from waffles.utils.integral.integral_utils import get_pulse_window_limits

beam_average_timetick = 15600
delta_beam_average_timetick = 200
if beam_average_timetick is not None:
    print('caio')
    adcs_array = mean_wf.adcs[beam_average_timetick-delta_beam_average_timetick: beam_average_timetick+delta_beam_average_timetick]
else: 
    print('nada')
    adcs_array = mean_wf.adcs

# Compute integration limits
limits = get_pulse_window_limits(adcs_array, 690, 0.3, 0, 0)
limits = list(limits)
print(limits)
if beam_average_timetick is not None:
    limits[0] = limits[0] + beam_average_timetick-delta_beam_average_timetick
    limits[1] = limits[1] + beam_average_timetick-delta_beam_average_timetick
    print(limits)

In [ ]:
fig = go.Figure()
y = mean_wf.adcs
x = list(range(len(mean_wf.adcs)))
fig.add_trace(go.Scatter(x=x, y=y, mode='lines', name='mean_wf'))

fig.update_layout(
    title='Mean Waveform',
    xaxis_title='Samples',
    yaxis_title='Amplitude',
    template='plotly_white'
)

fig.add_shape(
    type="line",
    x0=limits[0], x1=limits[0],
    y0=min(y), y1=max(y),
    line=dict(width=2, dash="dash", color="red")
)

fig.add_shape(
    type="line",
    x0=limits[1], x1=limits[1],
    y0=min(y), y1=max(y),
    line=dict(width=2, dash="dash", color="red")
)

fig.show()

In [ ]:
importlib.reload(integral_module)
from integral_computation_function import channel_integral_computation

importlib.reload(utils_plots_module)
from waffles.np04_analysis.lightyield_vs_energy.scripts.utils_plots import *

apa = 1
endpoint = 104
channel = 7
integration_analysis_label = 'integrator'
period = 'june'
grid_apa_FS = ChannelWsGrid(APA_map[apa], FS_wfset, compute_calib_histo=False)
# for endpoint in grid_apa_FS.ch_wf_sets.keys():
#     for channel in grid_apa_FS.ch_wf_sets[endpoint].keys():
#         grid_apa_FS.ch_wf_sets[endpoint][channel] = channel_integral_computation(grid_apa_FS.ch_wf_sets[endpoint][channel], period = period, integration_analysis_label = integration_analysis_label)

grid_apa_FS.ch_wf_sets[endpoint][channel] = channel_integral_computation(grid_apa_FS.ch_wf_sets[endpoint][channel], period = period, integration_analysis_label = integration_analysis_label, beam_average_timetick = 15600)

# plotting_overlap_wf_(grid_apa.ch_wf_sets[endpoint][channel], n_wf = 10, show = True, save = False, x_min=None, x_max=None, y_min=None, y_max=None, int_ll=grid_apa.ch_wf_sets[endpoint][channel].waveforms[0].analyses[integration_analysis_label].result['integration_limits'][0], int_ul=grid_apa.ch_wf_sets[endpoint][channel].waveforms[0].analyses[integration_analysis_label].result['integration_limits'][1], baseline=None, output_folder = 'output', deconvolution = False, analysis_label = integration_analysis_label)
plotting_overlap_wf_PEAK_NEW(grid_apa_FS.ch_wf_sets[endpoint][channel], n_wf= 10, show  = True, peak_bool = True, peak_beam_bool = True, analysis_label=integration_analysis_label, int_ll=grid_apa_FS.ch_wf_sets[endpoint][channel].waveforms[0].analyses[integration_analysis_label].result['integration_limits'][0], int_ul=grid_apa_FS.ch_wf_sets[endpoint][channel].waveforms[0].analyses[integration_analysis_label].result['integration_limits'][1])